In [1]:
#!python3.8 -m pip install fastcore

In [1]:
import sys
import time
import odrive
from odrive.enums import *
import fibre.libfibre
# from fastcore.nb_imports import *
from fastcore.all import *
import ipywidgets as widgets
import datetime
from IPython.display import display
import threading

In [2]:
PERFORM_TESTS = True

In [3]:
class OdriveMotorController:
    """
    Class for configuring an Odrive axis for a Hoverboard motor. 
    Only works with one Odrive at a time.
    """
    FIND_ODRIVE_RETRIES = 10 # Number of retries to find odrive after reset with 1 sec delay per try
    MOTOR_KV = 10.0 # Motor KV
    # Min/Max phase inductance of motor
    MIN_PHASE_INDUCTANCE = 0.0008 #Original: 0 : 0.0017 for up to 6va : 0.0012 for 7
    MAX_PHASE_INDUCTANCE = 0.0024 #Measured: 0.0019619388040155172   Original: 0.001
    # Min/Max phase resistance of motor
    MIN_PHASE_RESISTANCE = 0.77 #Original: 0.0
    MAX_PHASE_RESISTANCE = 0.92 #Measured: 0.8209025263786316 Original 0.5
    # Tolerance for encoder offset float
    ENCODER_IS_HALL = False
    ENCODER_OFFSET_FLOAT_TOLERANCE = 0.05 #Only relevant for hall sensors I think?
    
    def __init__(
        self, 
        selected_axis_num:int,
        find_odrive_max_retries:int=10
    ):
        """
        Initalizes HBMotorConfig class by finding odrive, erase its 
        configuration, and grabbing specified axis object.
        
        :param axis_num: Which channel/motor on the odrive your referring to.
        :type axis_num: int (0 or 1)
        """
        self.selected_axis_num = selected_axis_num
        self.selected_axis = None
        self.find_odrive_max_retries = find_odrive_max_retries
        
        self._find_odrive()
        
    def _find_odrive(self):
        """
        Connect to Odrive
        """
        odrive_found = False
        for i in range(self.find_odrive_max_retries):
            try:
                self.odrv = odrive.find_any()
                odrive_found = True
                break
            except fibre.libfibre.ObjectLostError:
                print('Odrive temporarily not found. Trying again....')
                time.sleep(.5)
        if odrive_found == False: raise Exception("Odrive Not Found")
        self.selected_axis = getattr(self.odrv, "axis{}".format(self.selected_axis_num))

In [4]:
@patch
def dump_errors(self:OdriveMotorController):
    dump_errors(self.odrv)

In [5]:
if PERFORM_TESTS: omc = OdriveMotorController(selected_axis_num=1)

In [6]:
@patch
def select_axis(self:OdriveMotorController, selected_axis_num:int=0):
    self.selected_axis_num = selected_axis_num
    self._find_odrive()

In [7]:
if PERFORM_TESTS:
    temp_orig_selected_axis_num = omc.selected_axis_num
    omc.select_axis(selected_axis_num=0)
    assert omc.selected_axis_num == 0
    assert omc.selected_axis == omc.odrv.axis0
    omc.select_axis(selected_axis_num=1)
    assert omc.selected_axis_num == 1
    assert omc.selected_axis == omc.odrv.axis1
    omc.select_axis(selected_axis_num=temp_orig_selected_axis_num)

In [8]:
@patch
def _wait_idle(self:OdriveMotorController):
    """
    Waits until axis is idle
    """
    time.sleep(.1)
    while self.selected_axis.current_state != AXIS_STATE_IDLE:
        time.sleep(.1)

In [9]:
@patch
def _save_config(self:OdriveMotorController):
    """
    Save the Odrive configuration and reboot
    """
    try:
        self.odrv.save_configuration()
    except fibre.libfibre.ObjectLostError:
        pass
    print("Configuration Saved.")
    self._find_odrive()
    self._wait_idle()

In [10]:
# if PERFORM_TESTS: omc._save_config()

In [11]:
@patch
def _reboot_odrive(self:OdriveMotorController):
    print('Rebooting Odrive...')
    try:
        self.odrv.reboot()
    except fibre.libfibre.ObjectLostError:
        pass
    self._find_odrive()
    self._wait_idle()

In [12]:
# if PERFORM_TESTS: omc._reboot_odrive()

In [13]:
def set_idle():
    omc.odrv.axis0.requested_state = AXIS_STATE_IDLE
    omc.odrv.axis1.requested_state = AXIS_STATE_IDLE

In [14]:
def set_velocity(velocity,enable=False):
    try:
        if type(velocity) in (int,float):
            v0 = float(velocity)
            v1 = float(velocity)
        elif type(velocity) in (list,tuple):
            assert len(velocity) == 2
            v0 = float(velocity[0])
            v1 = float(velocity[1])
        else:
            assert False,'improper input type'
        if enable:
            omc.odrv.axis0.controller.config.control_mode = CONTROL_MODE_VELOCITY_CONTROL
            omc.odrv.axis0.controller.config.input_mode = INPUT_MODE_VEL_RAMP
            omc.odrv.axis1.controller.config.control_mode = CONTROL_MODE_VELOCITY_CONTROL
            omc.odrv.axis1.controller.config.input_mode = INPUT_MODE_VEL_RAMP
        if ((v0 > 0. or v0 < 0.) or (v1 > 0. or v1 < 0.)) and omc.odrv.axis0.current_state != AXIS_STATE_CLOSED_LOOP_CONTROL and omc.odrv.axis1.current_state != AXIS_STATE_CLOSED_LOOP_CONTROL:
            omc.odrv.axis0.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL
            omc.odrv.axis1.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL
        omc.odrv.axis0.controller.input_vel = v0
        omc.odrv.axis1.controller.input_vel = v1
        if v0 == 0. and v1 == 0.:
            time.sleep(.5)
            set_idle()
    except:
        set_idle()
        print('Exception: idle successful')

In [37]:
set_velocity(0,enable=True)
# set_velocity((0,0))

In [38]:
set_idle()

In [17]:
def on_submit_func(*args,**kwargs):
    print(args)
    print(kwargs)

In [18]:
class Keyboard_Controller:
    VALID_KEYS = ['','a','s','d','w',' ']
    VELOCITY_UPPER_LIMIT = 2.
    VELOCITY_LOWER_LIMIT = -2.
    TURN_UPPER_LIMIT = 2.
    TURN_LOWER_LIMIT = -2.
    VELOCITY_DECAY = 0.02
    TURN_DECAY = 0.03
    VELOCITY_INCREMENT = .2
    TURN_INCREMENT = .1
    HEARTBEAT_TIMEOUT_SEC = 5
    def __init__(self):
        self.velocity = 0.
        self.turn_offset = 0.
        self.widget_control = widgets.Text()
        self.text_label = widgets.Label()
        self.velocity_slider = widgets.FloatSlider(min=self.VELOCITY_LOWER_LIMIT,max=self.VELOCITY_UPPER_LIMIT,description='Velocity')
        self.turn_slider = widgets.FloatSlider(min=self.TURN_LOWER_LIMIT,max=self.TURN_UPPER_LIMIT,description='Turning')
        self.v0 = 0.
        self.v1 = 0.
        self.last_heartbeat = datetime.datetime.now()
        self.exit_loop = True
        display(self.widget_control)
        display(self.text_label)
        display(self.velocity_slider)
        display(self.turn_slider)
        
    def emergency_shutdown(self):
        self.exit_loop = True
        set_idle()
        self.exit_loop = True
    
    def loop(self):
#             print(self.widget_control.value)
        if len(self.widget_control.value) > 1:
            k = self.widget_control.value[0]
        else:
            k = self.widget_control.value
        self.widget_control.value = ''
#             print(k)
        if k == '':
            pass
        else:
            self.last_heartbeat = datetime.datetime.now()
            if k == 'w':
                if 0. > self.velocity > (-1. * self.VELOCITY_INCREMENT) and self.velocity != 0.:
                    self.velocity = 0.
                else:
                    self.velocity += self.VELOCITY_INCREMENT
            elif k == 's':
                if self.VELOCITY_INCREMENT > self.velocity > 0. and self.velocity != 0.:
                    self.velocity = 0.
                else:
                    self.velocity -= self.VELOCITY_INCREMENT
            elif k == 'd':
                if 0. > self.turn_offset > (-1. * self.TURN_INCREMENT):
                    self.turn_offset = 0.
                else:
                    self.turn_offset += self.TURN_INCREMENT
            elif k == 'a':
                if self.TURN_INCREMENT > self.turn_offset > 0.:
                    self.turn_offset = 0.
                else:
                    self.turn_offset -= self.TURN_INCREMENT
            elif k == 'z':
                self.velocity = 0.
                self.turn_offset = 0.
            elif k == ' ':
                pass
            else: #any other key
                self.emergency_shutdown()
        if (datetime.datetime.now() - self.last_heartbeat).total_seconds() > self.HEARTBEAT_TIMEOUT_SEC and (self.velocity != 0. or self.turn_offset !=0.):
            self.emergency_shutdown()
        
        if self.velocity > self.VELOCITY_UPPER_LIMIT: self.velocity = self.VELOCITY_UPPER_LIMIT
        if self.velocity < self.VELOCITY_LOWER_LIMIT: self.velocity = self.VELOCITY_LOWER_LIMIT
        if self.turn_offset > self.TURN_UPPER_LIMIT: self.turn_offset = self.TURN_UPPER_LIMIT
        if self.turn_offset < self.TURN_LOWER_LIMIT: self.turn_offset = self.TURN_LOWER_LIMIT
        self.velocity_slider.value = self.velocity
        self.turn_slider.value = self.turn_offset

        v0 = self.velocity + self.turn_offset/2.
        v1 = self.velocity - self.turn_offset/2.
        if v0 > self.VELOCITY_UPPER_LIMIT: v0 = self.VELOCITY_UPPER_LIMIT
        if v0 < self.VELOCITY_LOWER_LIMIT: v0 = self.VELOCITY_LOWER_LIMIT
        if v1 > self.VELOCITY_UPPER_LIMIT: v1 = self.VELOCITY_UPPER_LIMIT
        if v1 < self.VELOCITY_LOWER_LIMIT: v1 = self.VELOCITY_LOWER_LIMIT

        if abs(self.velocity) < self.VELOCITY_DECAY:
            self.velocity = 0.
        elif self.velocity > 0.:
            self.velocity -= self.VELOCITY_DECAY
        elif self.velocity < 0.:
            self.velocity += self.VELOCITY_DECAY
        else:
            assert False,'bad velocity decay'
        if abs(self.turn_offset) < self.TURN_DECAY:
            self.turn_offset = 0.
        elif self.turn_offset > 0.:
            self.turn_offset -= self.TURN_DECAY
        elif self.turn_offset < 0.:
            self.turn_offset += self.TURN_DECAY
        else:
            assert False,'bad turn decay'

        if self.exit_loop == False: set_velocity((v0,v1))
        
        self.text_label.value = str((
            self.widget_control.value,
            k,
            round(self.velocity,2),
            round(self.turn_offset,2),
            round(v0,2),
            round(v1,2),
            round((datetime.datetime.now()-self.last_heartbeat).total_seconds(),2)),
#             round(self.odrv.)
        )
#         print(self.text_label.value)
    
    def _run_loop(self):
        self.last_heartbeat = datetime.datetime.now()
        self.exit_loop = False
        try:
            while self.exit_loop == False:
                time.sleep(.1)
                self.loop()
        except:
            print('Loop Exception')
            self.emergency_shutdown()
        self.emergency_shutdown()

    def run_loop(self):
        #thread = threading.Thread(target=work, args=(w,))
        thread = threading.Thread(target=self._run_loop)
        thread.start()

In [39]:
print('w-Fwd,s-Rev,a-Lft,d-Rgt,z-Off,space-HrtBt,anykey-estop')
kc = Keyboard_Controller()
kc.run_loop()

w-Fwd,s-Rev,a-Lft,d-Rgt,z-Off,space-HrtBt,anykey-estop


Text(value='')

Label(value='')

FloatSlider(value=0.0, description='Velocity', max=2.0, min=-2.0)

FloatSlider(value=0.0, description='Turning', max=2.0, min=-2.0)

In [34]:
omc.odrv.error, omc.odrv.axis0.error, omc.odrv.axis0.motor.error, omc.odrv.axis0.encoder.error

(0, 0, 0, 0)

In [35]:
omc.odrv.error, omc.odrv.axis1.error, omc.odrv.axis1.motor.error, omc.odrv.axis1.encoder.error

(0, 0, 0, 0)

In [22]:
omc.odrv.clear_errors()

In [23]:
# kc.emergency_shutdown()

In [ ]:
assert False

In [33]:
omc._reboot_odrive()

Rebooting Odrive...
